## Add an ID column to the products, accounts, and sales_teams CSV files

In [11]:
import pandas as pd 
import numpy as np 

# folder paths
original_file="C:\\Users\\kadla\\OneDrive\\Desktop\\CRM Sales opportunity Projet_SQL\\Original_Files\\"
modified_file="C:\\Users\\kadla\\OneDrive\\Desktop\\CRM Sales opportunity Projet_SQL\\Modified_files\\"

# list of CSV file names
cvs_file_name = ['accounts.csv',
                  'products.csv',
                  'sales_teams.csv']
for file in cvs_file_name:
    open_path = original_file+file
    df=pd.read_csv(open_path)
    #print(df.head())

    # define values for the new "id" column
    max_index = df.index.max()+1
    id_values = np.arange(1,max_index+1,1)

    # add new "id" column to the dataframe
    df.insert(loc=0,column = 'id' ,value=id_values)
    print(df.head())

    # save dataframe back to the CSV file
    save_path = modified_file + file
    df.to_csv(save_path, index=False)

   id           account     sector  year_established  revenue  employees  \
0   1  Acme Corporation  technolgy              1996  1100.04       2822   
1   2        Betasoloin    medical              1999   251.41        495   
2   3          Betatech    medical              1986   647.18       1185   
3   4        Bioholding    medical              2012   587.34       1356   
4   5           Bioplex    medical              1991   326.82       1016   

  office_location subsidiary_of  
0   United States           NaN  
1   United States           NaN  
2           Kenya           NaN  
3      Philipines           NaN  
4   United States           NaN  
   id       product series  sales_price
0   1     GTX Basic    GTX          550
1   2       GTX Pro    GTX         4821
2   3    MG Special     MG           55
3   4   MG Advanced     MG         3393
4   5  GTX Plus Pro    GTX         5482
   id        sales_agent           manager regional_office
0   1      Anna Snelling  Dustin Brinkma

## Create a dictionary with ID and full name for the products, accounts, and sales_teams CSV files



Goal: Prepare the data to replace records in sales_pipeline.csv

In [16]:
import csv

# dictionary names for each CSV file
dict_names = ['accounts_dict',
              'products_dict',
              'teams_dict']

for i in range(len(cvs_file_name)):
    
    file = cvs_file_name[i]
    dict_name = dict_names[i]
    
    #read modified csv files (with id's)
    with open(modified_file + file) as csvfile:
        reader = csv.reader(csvfile)
        
        # skip header
        next(reader, None)
        
        temporary_dict = {}

        for row in reader:
            # key: full name; value: assigned ID
            temporary_dict[row[1]] = row[0]
        
        # save temporary dictionary data to the final dictionary
        globals()[dict_name] = temporary_dict

print(f'Dictionary example: {products_dict}')

Dictionary example: {'GTX Basic': '1', 'GTX Pro': '2', 'MG Special': '3', 'MG Advanced': '4', 'GTX Plus Pro': '5', 'GTX Plus Basic': '6', 'GTK 500': '7'}


## Check for values in the dictionary that do not match the full names in sales_pipeline.csv

In [17]:
sales_pipeline_path = f'{original_file}sales_pipeline.csv'
sales_df = pd.read_csv(sales_pipeline_path)

# sales_pipeline column and corresponding dictionary
data = {'product': products_dict,
        'account': accounts_dict,
        'sales_agent': teams_dict}

for column, dict in data.items():
    print(f'Sales_pipeline column - {column}:')

    # check for non matching values in sales_pipeline column
    x = sales_df[~sales_df[column].isin(dict.keys())][column].unique()
    
    if len(x) != 0:
        print(f'Non matching values in sales_pipeline column: {x}.')        

        #check for non matching values in the dictionary 
        for keys in dict.keys():
            y = sales_df[column].unique()
            if keys not in y:
                print(f'Non matching values in the dictionary: {keys}.')
    
    else:
        print(f'No missing values in the dictionary.')

Sales_pipeline column - product:
Non matching values in sales_pipeline column: ['GTXPro'].
Non matching values in the dictionary: GTX Pro.
Sales_pipeline column - account:
Non matching values in sales_pipeline column: [nan].
Sales_pipeline column - sales_agent:
No missing values in the dictionary.


##### Conclusion: there's a misspelled product name 'GTX Pro' in the dictionary.

### Modify misspelled values in the products dictionary

In [18]:
# replace 'GTX Pro' to 'GTXPro'
products_dict['GTXPro'] = products_dict['GTX Pro']

del products_dict['GTX Pro']

### Replace full names in sales_pipeline.csv with IDs

In [19]:
for column, dict in data.items():
    sales_df[column] = sales_df[column].map(dict)

In [20]:
#preview of the modified sales_pipeline dataframe
sales_df

,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value
0,1C1I7A6R,5,6,9,Won,2016-10-20,2017-03-01,1054.0
1,Z063OYW0,10,2,39,Won,2016-10-25,2017-03-11,4514.0
2,EC4QE1BX,10,3,9,Won,2016-10-25,2017-03-07,50.0
3,MV1LWRNH,5,1,11,Won,2016-10-25,2017-03-09,588.0
4,PE84CX4O,33,1,35,Won,2016-10-25,2017-03-02,517.0
...,...,...,...,...,...,...,...,...
8795,9MIWFW5J,3,4,NaN,Prospecting,NaN,NaN,NaN
8796,6SLKZ8FI,3,4,NaN,Prospecting,NaN,NaN,NaN
8797,LIB4KUZJ,3,4,NaN,Prospecting,NaN,NaN,NaN
8798,18IUIUK0,3,4,NaN,Prospecting,NaN,NaN,NaN


### Save changes to sales_pipeline.csv

In [21]:
sales_df.to_csv(f"{modified_file}sales_pipeline.csv", index=False)